In [ ]:
'''해야할 일
   1)스피어만 상관분석은 비모수 통계기법으로 비모수 통계기법 파일로 옮긴다.
   2)정준상관분석 유의성 검정이 존재하는 지 조사한다.
   3)다중상관분석과 정준상관분석 Code 조사하기
'''

In [12]:
import pandas as pd
data = pd.read_csv('data/Ashopping.csv', encoding='cp949')
cca = pd.read_csv('data/CCA.csv', encoding='cp949')

## Chapter 7. Correlation Analysis
두 개 이상의 변수들간 상관관계를 파악하기 위해 수행하는 방법이다.
- 양측검정 시 상관분석의 귀무가설은 "변수들이 서로 독립이다."이고 대립가설은 "변수들이 서로 독립이지 않다. 변수간 선형적인 관계가 존재한다."이다.
  $$H_0 : r = 0,~ H_1 : r \neq 0$$
- 일반적으로 상관분석은 변수간 상관계수 값이 모호할 때 그 값이 믿을 만한지 검정하는 것으로 <br>
  단측검정을 수행해 분석 결과에서 더 많은 정보를 얻어내고자 한다.<br>
  상관계수가 양수였다면 $H_0 : r > 0, ~ H_1 : r < 0$로 가설을 세우고 음수였다면 $H_0 : r < 0, ~ H_1 : r > 0$로 가설을 세운다.
- 수치형 변수간 상관관계를 검정하고자 하면 검정에 사용할 상관계수로 Pearson Correlation을 사용하고 <br>
  서열척도 변수간 상관관계를 검정하고자 하면 Spearman Correlation을 사용한다.
- 상관관계를 파악하고 싶은 변수의 개수가 2개이면 Simple Correlation Analysis, 3개 이상이면 Multiple Correlation Analysis로 구분한다.
- 다중상관분석을 통해 여러 변수간 상호연관성을 한번에 파악할 수 있으나 두 변수만의 순수한 상관관계를 파악할 수 없다.<br>
  다중선형회귀모형에서 회귀계수의 유의를 따지듯 다중상관관계분석에서도 2개의 변수 이외의 변수들을 고정시키고 두 변수만의 순수한 상관관계를 파악하는 분석을 `Partial Correlation Analysis(부분상관관계분석)`라 한다.
- 변수가 아닌 변수 그룹간 상관관계를 파악하고 싶을 때 수행하는 분석으로 정준상관분석이 있다.

In [ ]:
# 예시 1) 세 수치형 변수간 상관분석을 수행한다.

In [7]:
# - 1. 세 변수간 상관관계를 출력한다.
#    :출력된 상관계수 모두 0.7 미만으로 상관분석을 수행해야 한다.
data[['총_매출액','방문빈도','구매_카테고리_수']].corr(method='pearson')

,총_매출액,방문빈도,구매_카테고리_수
총_매출액,1.000000,0.631171,0.308347
방문빈도,0.631171,1.000000,0.395861
구매_카테고리_수,0.308347,0.395861,1.000000


In [6]:
# - 2. 피어슨 단순상관관계분석으로 세 번에 걸쳐 분석한다.
#    :p-value에 따라 산출된 상관계수는 믿을 만하며
#    총매출액과 방문빈도 사이 양의 상관관계가, 총매출액과 구매카테고리수, 방문빈도와 구매카테고리수의 상관관계는 미미하다 결론지을 수 있다.
from scipy.stats import pearsonr
print('총매출액 & 방문빈도:', pearsonr(data.총_매출액, data.방문빈도))
print('총매출액 & 구매카테고리수:', pearsonr(data.총_매출액, data.구매_카테고리_수))
print('방문빈도 & 구매카테고리수:', pearsonr(data.방문빈도, data.구매_카테고리_수))

총매출액 & 방문빈도: (0.6311706453193395, 3.051960449687332e-112)
총매출액 & 구매카테고리수: (0.3083469132396712, 1.8121422756812584e-23)
방문빈도 & 구매카테고리수: (0.39586072218464485, 7.345432750388936e-39)


In [ ]:
# 예시 2) 두 서열척도 변수간 상관분석을 수행한다.

In [11]:
# - 1. 스피어만 단순상관관계분석을 수행한다.(scipy.stats의 spearmanr 함수는 수치형변수가 입력되면 자동으로 서열척도로 변환해 분석한다.)
#   :믿을만한 상관계수를 구해보니 -0.49로 
#    방문빈도가 높은 고객은 1회 평균매출액이 적고, 방문빈도가 작은 고객은 1회 평균매출액이 크다고 밝혀졌다.
from scipy.stats import spearmanr
spearmanr(data['1회_평균매출액'], data['방문빈도'])

SpearmanrResult(correlation=-0.4988411248473936, pvalue=4.929293870381245e-64)

### Partial Correlation Analysis(부분상관관계분석)
다중상관관계 분석을 통해 구한 상관계수는 변수들의 획일적인 방향을 의미하는 것으로 상관관계를 따져본 변수들이 최적의 조합인지, 조합된 변수들에 영향을 미치는 compound 변수로 인해 상관관계가 파생되는 것 아닌지 따져보아야 한다. 
- 2개의 변수 이외의 변수들에 의해 선형적으로 설명되는 부분을 제거하고 남은 잔차들간의 상관계수를 구한다.<br>
  $z(\text{:제3의 변수}) \rightarrow x, z \rightarrow y$인 단순회귀분석에서의 잔차를 구한 후 두 잔차들간 상관계수를 구하면 두 변수 $x, y$간 편상관계수가 된다.
  $$r_{xy \cdot x} = \frac{r_{xy} - r_{xz} \cdot r_{yz}}{\sqrt{1-r_{xz}^2} \cdot \sqrt{\small{1-r_{yz}^2}} }$$
- 부분상관관계분석의 가설은 아래와 같다.
  <center>$H_0: z$가 통제된 상황에서 $x, y$는 유의한 상관관계를 갖지 않는다.&ensp;
  $H_1: z$가 통제된 상황에서 $x, y$는 유의한 상관관계를 갖는다.&ensp;&ensp;&ensp;&ensp;</center>

In [ ]:
# 부분상관관계분석을 위해 패키지 설치가 필요하다.
# !pip install pingouin

In [ ]:
# 예시 3) 위의 예시 1에서 확인한 총매출액과 방문빈도간 순수한 상관관계를 확인한다.

In [8]:
# - 1. 부분상관분석을 수행한다.
#    :구매카테고리수 변수를 영향을 제거하자 상관계수가 줄어들었다. 
#     즉, 구매카테고리수는 총매출액, 방문빈도와 낮은 상관관계를 갖으나, 영향력이 적으나
#     총매출액과 방문빈도간 관계에 직접적인 영향을 주기에 함께 고려함이 바람직하다.
from pingouin import partial_corr
partial_corr(data=data, x='총_매출액', y='방문빈도', covar='구매_카테고리_수')

,n,r,CI95%,p-val
pearson,1000,0.582794,"[0.54, 0.62]",6.223289e-92


### Canonical Correlation Analysis(정준상관분석, CCA)
다수의 변수가 조합되어 만들어진 변수 군집간 상관관계를 파악해 변수 군집간 상관성을 가장 잘 표현해주는 요인변수들의 선형결합을 찾는 분석 방법이다. 
- 모든 변수는 연속형 변수임을 가정한다. 범주형 변수를 사용하고자 한다면 더미변수화(One-Hot Encoding)해야 한다.
- 정준상관분석에서 사용하는 용어는 아래와 같다.
  $$X_1 = \alpha' x_{1[1\times p]} = \alpha_1x_{11} + \dots + \alpha_px_{1p},~ X_2 = \beta' x_{2[1\times q]} = \beta_1x_{21} + \dots + \beta_qx_{2q}$$
  - 정준변수($X_1, X_2$): 복수의 변수로 이루어진 변수 그룹
  - 정준상관계수($\rho = corr(X_1, X_2)$): 정준변수간 상관계수
  - 정준계수($\alpha_k, \beta_k$): 정준상관계수를 최대화하는 선형결합에서 개별변수들의 계수
- 정준상관분석은 정준상관계수, 정준변수를 추정하는 것 외에도 정준적재량과 교차적재량으로 두 변수 그룹간의 관계에 대한 각 변수의 상대적 영향력을 확인해 최소 요인변수들의 선형결합을 찾길 목적한다.
  - 정준적재량: 정준변수와 해당 정준변수를 구성하는 측정변수들 사이의 상관계수(ex|$X$와 $x_k$간 상관계수)
  - 교차적재량: 정준변수와 대립하는 정준변수의 측정변수들 사이의 상관계수(ex|$X$와 $y_k$간 상관계수)

In [ ]:
# 예시 4) 제품만족도에 해당하는 가격, 품질, 디자인과 
#         매장만족도에 해당하는 직원서비스, 매장시설, 고객관리 변수에 대해 정준상관분석을 수행한다.

In [13]:
# - 1. 정준변수를 선언한다.
X1, X2 = cca[['품질','가격','디자인']], cca[['직원 서비스','매장 시설','고객관리']]

In [14]:
# - 2. 정준상관분석을 수행한다.
from sklearn.cross_decomposition import CCA
X1_cca, X2_cca = CCA(n_components=1).fit_transform(X1, X2)
X1_cca, X2_cca = X1_cca.flatten(), X2_cca.flatten()

In [15]:
#   :제1 정준상관계수는 0.77로 강한 양의 상관계수이며 유의수준 0.01에서 유의하다.
from scipy.stats import pearsonr
print('제1 정준상관계수:', pearsonr(X1_cca, X2_cca)) 

제1 정준상관계수: (0.7717687943183484, 0.0012226670192390231)


In [16]:
#  :정준적재량을 통해 X1에선 가격과 디자인이, X2에선 직원 서비스와 고객관리가 중요 변수임을 확인할 수 있으며
#   교차적재량을 통해 파악한 상세 영향요인은 X1 중 X2에 영향력이 큰 변수는 품질과 디자인이, 
#   X2 중 X1에 영향력이 큰 변수는 매장 시설과 고객관리라고 확인되었다.
from numpy import corrcoef
print(f'정준변수 X의 정준적재량 -> {X1.columns.values}:', corrcoef(X1_cca.T, X1.T)[0][1:])
print(f'정준변수 X의 교차적재량 -> {X1.columns.values}:', corrcoef(X1_cca.T, X2.T)[0][1:])
print(f'정준변수 Y의 정준적재량 -> {X2.columns.values}:', corrcoef(X2_cca.T, X2.T)[0][1:])
print(f'정준변수 Y의 교차적재량 -> {X2.columns.values}:', corrcoef(X2_cca.T, X1.T)[0][1:])

정준변수 X의 정준적재량 -> ['품질' '가격' '디자인']: [0.35045604 0.77461847 0.55191153]
정준변수 X의 교차적재량 -> ['품질' '가격' '디자인']: [ 0.70598452 -0.0438384   0.5889048 ]
정준변수 Y의 정준적재량 -> ['직원 서비스' '매장 시설' '고객관리']: [ 0.91476168 -0.0568025   0.76305858]
정준변수 Y의 교차적재량 -> ['직원 서비스' '매장 시설' '고객관리']: [0.27044132 0.5978383  0.42592516]
